In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
from cal import compute_scores_for_multiple_projects
from get_material import get_materials
from ml_models.ml_pipeline import get_n_days_forecast, get_num_workers

from cal import compute_scores_for_multiple_projects


def pipeline(projects_example, dynamic_weights, OPENAI_API_KEY= os.getenv('OPENAI_API_KEY')):
    res = compute_scores_for_multiple_projects(
        projects_example,
        dynamic_weights,
        OPENAI_API_KEY
    )
    print(res)

    
    

projects_example = [
    {
        "projectId": "101",
        "tasks": ["Build elevated track", "Install stations", "Lay electrical lines"],
        "timeline": 18,
        "budget": 500,
        "location": "Urban Metro Region"
    },
    {
        "projectId": "202",
        "tasks": [
            "Clear and grade existing roadway",
            "Lay asphalt and expand lanes",
            "Install drainage and stormwater systems"
        ],
        "timeline": 12,
        "budget": 450,
        "location": "Suburban / Semi-urban"
    },
    {
        "projectId": "203",
        "tasks": [
            "Conduct geotechnical surveys and soil testing",
            "Build foundation piles and piers",
            "Assemble steel beams and girders"
        ],
        "timeline": 18,
        "budget": 750,
        "location": "Riverine / Rural Region"
    }
]

dynamic_weights = {
    "w_cost_benefit": 0.25,
    "w_socio_impact": 0.30,
    "w_risk_resilience": 0.20,
    "w_strategic_align": 0.25
}

pipeline(projects_example, dynamic_weights)
    

c:\Users\VEMKATESH R\Desktop\aurigo\aurigo-hackathon\get_material.py:40: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4o-mini")


NotImplementedError: 

In [2]:
from langchain.llms import OpenAI
from langchain.tools import Tool
from langchain.agents import initialize_agent, Tool
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain_community.tools import TavilySearchResults
from pydantic import BaseModel
from typing import List
import tavily

import os
import getpass
from dotenv import load_dotenv

load_dotenv()

TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")



if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API key:\n")
    

tool = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=True,
)


if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")
llm = ChatOpenAI(model="gpt-4o-mini")


class MaterialBase(BaseModel):
    material: str
    price: int

class MaterialResponse(BaseModel):
    data: list[MaterialBase]

prompt = ChatPromptTemplate(
    [
         ("system", """You are an expert construction planner with access to a vast knowledge base and real-time web search capabilities. The response must be in below JSON format strictly without any explanation and comments:
        {{
            "data": [
                {{
                    "material": "Material 1", 
                    "price": 1000
                }},
                {{
                    "material": "Material 2", 
                    "price": 1500
                }},
                {{
                    "material": "Material 3", 
                    "price": 2000
                }}
                // Add more materials and their prices as needed
            ]
        }}"""),
        ("human", "{user_input}"),
        ("placeholder", "{messages}"),
    ]
)


llm_with_tools = llm.bind_tools([tool])
llm_chain = prompt | llm_with_tools
parser = PydanticOutputParser(pydantic_object=MaterialResponse)

@chain
def tool_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    tool_msgs = tool.batch(ai_msg.tool_calls, config=config)
    response = llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)
    return response

    

def get_materials(user_input:str):
    response = tool_chain.invoke(user_input)
    parsed_response = parser.parse(response.content)
    return parsed_response.model_dump()


NotImplementedError: 